In [8]:
import scanpy

adata = scanpy.read_h5ad("pbmc_multimodal.h5ad")
dataset=adata.obs

C:\Users\Yi Rui Cui\AppData\Local\Programs\Python\Python36\lib\site-packages\anndata\compat\__init__.py:183: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  FutureWarning,


# Some Outputs that might be helpful

In [2]:
#print(adata.X[0])
print(len(adata.obs_names.tolist()))
print(len(adata.var_names.tolist()))
print(adata.shape)
print(adata.obs.keys())
print(adata.var.keys())
print(adata.obs.shape)
print(adata.var.shape)


161764
20729
(161764, 20729)
Index(['nCount_ADT', 'nFeature_ADT', 'nCount_RNA', 'nFeature_RNA',
       'orig.ident', 'lane', 'donor', 'time', 'celltype.l1', 'celltype.l2',
       'celltype.l3', 'Phase', 'nCount_SCT', 'nFeature_SCT'],
      dtype='object')
Index(['features'], dtype='object')
(161764, 14)
(20729, 1)


# Creating the base dataset for the model (Preprocessing)


In [9]:
print(dataset.shape)
print(dataset.keys())
#Maybe also drop celltype.l2 and celltype.l3
cleanedDataset=dataset.drop(['time','donor','lane','orig.ident'],axis=1)
print(cleanedDataset.keys())
print(cleanedDataset.shape)
print(cleanedDataset.head())


(161764, 14)
Index(['nCount_ADT', 'nFeature_ADT', 'nCount_RNA', 'nFeature_RNA',
       'orig.ident', 'lane', 'donor', 'time', 'celltype.l1', 'celltype.l2',
       'celltype.l3', 'Phase', 'nCount_SCT', 'nFeature_SCT'],
      dtype='object')
Index(['nCount_ADT', 'nFeature_ADT', 'nCount_RNA', 'nFeature_RNA',
       'celltype.l1', 'celltype.l2', 'celltype.l3', 'Phase', 'nCount_SCT',
       'nFeature_SCT'],
      dtype='object')
(161764, 10)
                     nCount_ADT  nFeature_ADT  nCount_RNA  nFeature_RNA  \
L1_AAACCCAAGAAACTCA      7430.0           221     10823.0          2915   
L1_AAACCCAAGACATACA      5949.0           211      5864.0          1617   
L1_AAACCCACAACTGGTT      6547.0           217      5067.0          1381   
L1_AAACCCACACGTACTA      3508.0           207      4786.0          1890   
L1_AAACCCACAGCATACT      6318.0           219      6505.0          1621   

                    celltype.l1 celltype.l2 celltype.l3 Phase  nCount_SCT  \
L1_AAACCCAAGAAACTCA        Mono

# Imports and sklearn version (useful for debugging purposes)

In [16]:
import sklearn
from sklearn.model_selection import train_test_split
print(sklearn.__version__)

0.24.2
(1000, 20)
(1000,)
[ -8.52381793   5.24451077 -12.14967704  -2.92949242   0.99314133
   0.67326595  -0.38657932   1.27955683  -0.60712621   3.20807316
   0.60504151  -1.38706415   8.92444588  -7.43027595  -2.33653219
   1.10358169   0.21547782   1.05057966   0.6975331    0.26076035]
0


# Splitting preprocessed dataset into X (samples) and y (classes)

In [90]:
import pandas as pd
print(cleanedDataset.shape)
classes = cleanedDataset[['celltype.l1']]
print(classes.shape)
print(pd.get_dummies(classes).shape)
print(pd.get_dummies(classes).head(1))
classes=pd.get_dummies(classes)
samples=cleanedDataset.drop(['celltype.l1','celltype.l2','celltype.l3'],axis=1)
print(samples.shape)
print(pd.get_dummies(samples).shape)
print(pd.get_dummies(samples).head(1))
samples=pd.get_dummies(samples)

(161764, 10)
(161764, 1)
(161764, 8)
                     celltype.l1_B  celltype.l1_CD4 T  celltype.l1_CD8 T  \
L1_AAACCCAAGAAACTCA              0                  0                  0   

                     celltype.l1_DC  celltype.l1_Mono  celltype.l1_NK  \
L1_AAACCCAAGAAACTCA               0                 1               0   

                     celltype.l1_other  celltype.l1_other T  
L1_AAACCCAAGAAACTCA                  0                    0  
(161764, 7)
(161764, 9)
                     nCount_ADT  nFeature_ADT  nCount_RNA  nFeature_RNA  \
L1_AAACCCAAGAAACTCA      7430.0           221     10823.0          2915   

                     nCount_SCT  nFeature_SCT  Phase_G1  Phase_G2M  Phase_S  
L1_AAACCCAAGAAACTCA      6380.0          2548         1          0        0  


# Creating X_train, X_test, y_train and y_test

In [91]:
print("Splitting data into training and test set")
X_train, X_test, y_train, y_test = train_test_split(
    samples, classes, test_size=0.3, random_state=42)


Splitting data into training and test set


# Dummy Classifier for baseline comparison


In [92]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score

dummy = DummyClassifier(random_state=0, strategy='stratified')
print("shapes of original X and y\t{}\t\t{}".format(samples.shape, classes.shape))
print("shapes of test X and y\t\t{}\t\t{}".format(X_test.shape, y_test.shape))
print("shapes of training X and y\t{}\t\t{}".format(X_train.shape, y_train.shape))
print("Beginning fitting of model")
dummy.fit(X_train, y_train)
print("Model has been fit, starting prediction")
y_pred = dummy.predict(X_test)
print("Accuracy:\t", float(accuracy_score(y_test, y_pred)))
print("F1-Score:\t", f1_score(y_test, y_pred, average='macro'))


shapes of original X and y	(161764, 9)		(161764, 8)
shapes of test X and y		(48530, 9)		(48530, 8)
shapes of training X and y	(113234, 9)		(113234, 8)
Beginning fitting of model
Model has been fit, starting prediction
Accuracy:	 0.08961467133731713
F1-Score:	 0.12604227524023495


# The Final Model

In [103]:
from sklearn.ensemble import RandomForestClassifier


clf = RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=0)
print("shapes of original X and y\t{}\t\t{}".format(samples.shape, classes.shape))
print("shapes of test X and y\t\t{}\t\t{}".format(X_test.shape, y_test.shape))
print("shapes of training X and y\t{}\t\t{}".format(X_train.shape, y_train.shape))
print("Beginning fitting of model")
clf.fit(X_train, y_train)
print("Model has been fit, starting prediction")
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
print(y_pred.shape)
print("Accuracy:\t{}\t\tF1-Score:\t{}".format(accuracy, f1))
print(y_pred)

shapes of original X and y	(161764, 9)		(161764, 8)
shapes of test X and y		(48530, 9)		(48530, 8)
shapes of training X and y	(113234, 9)		(113234, 8)
Beginning fitting of model
Model has been fit, starting prediction
(48530, 8)
Accuracy:	0.5983103235112301		F1-Score:	0.5341668474494423
[[0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
import pickle
pickle.dump(clf,open("model.pkl","wb"))